# APOL1 expression vs age

In [2]:
#Import modules
import pandas as pd
import numpy as np
import time as tm
import os
import dxdata
from bgen import BgenReader

In [ ]:

#Read the population assignment data:
ans = pd.read_csv("/mnt/project/Datasets/Ancestry/panukbb_ancestry.txt", sep="\t", index_col=0)
ans.rename(columns={"pop":"population"}, inplace=True) #original bad naming..
ans.set_index("individual_id", inplace=True)
ans = ans[~ans.index.duplicated(keep="first")]
ans.index = ans.index.astype(str).astype(object)

#Age, sex:
DATASET_ID = "REDACTED"
dataset = dxdata.load_dataset(id=DATASET_ID)
participant = dataset['participant']
field_age = participant.find_field(title="Age at recruitment")
field_sex = participant.find_field(title="Sex")
field_eid = participant.find_field(name="eid")
agesex = participant.retrieve_fields(engine=dxdata.connect(), fields=[field_eid, field_age, field_sex], coding_values="replace").toPandas()
agesex.set_index("eid", inplace=True)
agesex.columns = ["age", "sex"]
agesex["sex"] = agesex.sex.str.replace("Female","0").str.replace("Male","1").astype(int)

#v1, g1 and g2:
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.sample"
sns = pd.read_csv(fn, sep=" ")
sns = sns.iloc[1:,:]
sns = sns.ID_1

fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.bgen"
bfile = BgenReader(fn)
v = bfile.with_rsid("22:36265284:G:A")
print (len(v)) #Yes only one variant, good
v1 = pd.Series(v[0].alt_dosage)
v1.index = sns

g2v = bfile.with_rsid("22:36265995:AATAATT:A")
print (len(g2v)) #Yes only one variant, good
g2 = pd.Series(g2v[0].alt_dosage)
g2.index = sns

g1mv = bfile.with_rsid("22:36265988:T:G")
print (len(g2v)) #Yes only one variant, good
g1m = pd.Series(g1mv[0].alt_dosage)
g1m.index = sns

/tmp/ipykernel_82/1921754713.py:22: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sns = pd.read_csv(fn, sep=" ")


1
1
1


In [4]:
#APOL1 expression:
olinkdata = dataset['olink_instance_0']
field_apol1 = olinkdata.find_field(name="apol1")
apol1 = olinkdata.retrieve_fields(engine=dxdata.connect(), fields=[field_eid]+[field_apol1], coding_values="replace")
apol1 = apol1.toPandas().set_index("eid").squeeze()
apol1 = apol1[~apol1.isna()]

In [32]:
#make it a df:
apol1.index = apol1.index.astype(int)
ans.index = ans.index.astype(int)
agesex.index = agesex.index.astype(int)
#df = pd.concat([v1, g1m, g2, ans["population"], apol1], axis=1)
df = pd.concat([agesex, ans["population"], apol1], axis=1) #No genotype needed for now

In [33]:
#EUR and AFR separately
dfp = df[~df.apol1.isna()]
dfe = dfp[dfp.population=="EUR"]
dfa = dfp[dfp.population=="AFR"]

In [38]:
from scipy.stats import linregress
slope, intercept, r, p, se = linregress(dfp.age, dfp.apol1)
print (slope)
print (p)

-0.00965871012819904
6.647050117463741e-146
